# Review: Django

Django is a well-known web framework, and I guess the best part of Django is intuitive and object-oriented interfaces on database work. Followings are CRUD examples of `User` objects in Django:

```python
# create a user
user = User.objects.create(name="gh", password="1234")

# query all users
all_users = User.objects.all()

# filter users
all_kims = User.objects.filter(name__endswith="kim")

# query a user
user = User.objects.get(name="gh")

# update
user.name = "gkim"
user.save()

# delete
user.delete()
```

The point is `objects`, which is a manager to CRUD objects on database table. Actually, a manager is also an API component itself. For example, the only thing to make a *sign-up* API component is wrapping a manager to let it request and response data:

```python
class SignUp(API):
    def post(self, request):
        user, created = User.objects.get_or_create(
            name = request.data["name"],
            password = request.data["password"]
        )
        if not created:
            # if the user already exists
            return HTTP.403
        return HTTP.201
```

# Django-like backend for us

I've tried to rearrange our XAI backend workflow to Django one in this demo.

The XAI workflow has some key object types:
- `Dataset`: A user has a dataset and its subsets (samples) to be explained
- `Model`: A user has a model to be explained
- `Project`: An object containing multiple datasets (m), multiple models (n), and finally, multiple experiments (m x n)
    - What I actually want to do is defining a proj by a "dataset" (one-to-one relationship btw a proj and a dataset)
    - and then registering multiple (m) dataset-available preprocesses to the proj, instead of multiple datasets
    - This is actually what `wandb` or `mlflow` also does (`Preprocess` is one of `Artifact`)
    - However, I used `Dataset` instead of `Preprocess`, just for brief sketch and my convenience
- `Experiment` (or `Task`): A combination of a single dataset and a single model to be explained
- `Explainer`: An object to explain an experiment
- `Explanation`: Results from an explainer on an experiment

In this demo, I made `Dataset`, `Model`, `Project`, and `Task` (equivalent to `Experiment`) and did not explicitly make `Explainer` and `Explanation`, not yet. All these are one of (dictionary-like) dataclass consisting of its id and configs. Just for simplicity, the only a few configs are included in this demo:
- `Dataset`: [`id`, `name`, `uri`, `origin`]
- `Model`: [`id`, `name`, `uri`, `origin`]

They are working as follows:

## Overview: demo

### Object

In [2]:
import os
from pnpxai.client.objects import Dataset, Model, Project, Task

Just for checking how they look like, let's see `Dataset` object.

In [6]:
# create a dataset object without writing it in the database
dataset = Dataset(name="my_dataset", uri="some_uri")

In [9]:
print("dataset: ", dataset)
print("dataset.id: ", dataset.id)
print("dataset.name: ", dataset.name)
print("dataset.uri: ", dataset.uri)

dataset:  <Dataset: my_dataset>
dataset.id:  None
dataset.name:  my_dataset
dataset.uri:  some_uri


'to_dict' and 'to_tuple' methods will be helpful. You can find that I've set "torch" as a default value for "origin" field.

In [12]:
print("dataset.to_dict(): ", dataset.to_dict())
print("dataset.to_tuple(): ", dataset.to_tuple())

dataset.to_dict():  {'id': None, 'name': 'my_dataset', 'uri': 'some_uri', 'origin': 'torch'}
dataset.to_tuple():  (None, 'my_dataset', 'some_uri', 'torch')


### ObjectManager

Each object has its manager as a classmethod, named with "objects". The manager creates, reads, updates and deletes the object on database.

In [13]:
print("Dataset.objects: ", Dataset.objects)

Dataset.objects:  <pnpxai.client.manager.ObjectManager object at 0x7ffa16e9df60>


The manager's method "all" queries all objects in the database. Since we do not create any object yet, no results are queried as followings:

In [14]:
print("Dataset.objects.all(): ", Dataset.objects.all())
print("Model.objects.all(): ", Model.objects.all())
print("Project.objects.all(): ", Project.objects.all())
print("Task.objects.all(): ", Task.objects.all())

Dataset.objects.all():  []
Model.objects.all():  []
Project.objects.all():  []
Task.objects.all():  []


Now, you can find a file store named with "xaistore" is created on your current working directory. If you run some methods of the manager, the file store is automatically created and a database file named with "db.json" in the store. The database consists of tables for each object types and relationship tables between objects. This database is just a toy for demo and it should be fixed or replaced to more strict one.

## The XAI Workflow in demo

Please make sure the outputs from "setup_for_notebooks.ipynb" are in your current working directory.

### 1. User creates inputs: a dataset and a model

User registers his/her own inputs, a dataset and a model, to our system. I suppose that the user has serialized inputs in this case.

#### Dataset

In [16]:
dataset_uri = os.path.abspath('imagenet_sample.pt')

# the method `get_or_create` gets an obj if it exists else creates, and return obj and bool of created
dataset, created = Dataset.objects.get_or_create(name="first_dataset", uri=dataset_uri)

Now, you can find the dataset obj was written in the database, using the manager.

In [17]:
Dataset.objects.all()

[<Dataset: first_dataset>]

And the created obj looks like following.

In [18]:
print("dataset: ", dataset)
print("dataset.id: ", dataset.id)
print("dataset.name: ", dataset.name)
print("dataset.uri: ", dataset.uri)
print("dataset.origin: ", dataset.origin)
print("dataset.to_dict(): ", dataset.to_dict())
print("dataset.to_tuple(): ", dataset.to_tuple())
print("dataset.load(): ", dataset.load())
print("created: ", created)

dataset:  <Dataset: first_dataset>
dataset.id:  0
dataset.name:  first_dataset
dataset.uri:  /home/gkim/Projects/pnpxai-demo/notebooks/imagenet_sample.pt
dataset.origin:  torch
dataset.to_dict():  {'id': 0, 'name': 'first_dataset', 'uri': '/home/gkim/Projects/pnpxai-demo/notebooks/imagenet_sample.pt', 'origin': 'torch'}
dataset.to_tuple():  (0, 'first_dataset', '/home/gkim/Projects/pnpxai-demo/notebooks/imagenet_sample.pt', 'torch')
dataset.load():  <pnpxai.samples.datasets.ImageNetSample object at 0x7ffaf18455d0>
created:  True


#### Model

By the same way, user registers a model.

In [19]:
model_uri = os.path.abspath('resnet50.pt')
model, created = Model.objects.get_or_create(name="first_model", uri=model_uri)

In [20]:
Model.objects.all()

[<Model: first_model>]

In [21]:
print("model: ", model)
print("model.id: ", model.id)
print("model.name: ", model.name)
print("model.uri: ", model.uri)
print("model.origin: ", model.origin)
print("model.to_dict(): ", model.to_dict())
print("model.load(): ", model.load().__class__)
print("created: ", created)

model:  <Model: first_model>
model.id:  0
model.name:  first_model
model.uri:  /home/gkim/Projects/pnpxai-demo/notebooks/resnet50.pt
model.origin:  torch
model.to_dict():  {'id': 0, 'name': 'first_model', 'uri': '/home/gkim/Projects/pnpxai-demo/notebooks/resnet50.pt', 'origin': 'torch'}
model.load():  <class 'torchvision.models.resnet.ResNet'>
created:  True


Additionally, I've tried to make some methods about `ModelDetector` or `ExplainerRecommender` in task object. These methods are implemented in a task method `run_all_applicables` in step 4.

In [39]:
print("model.is_cam_applicable(): ", model.is_cam_applicable())
print("model.is_lrp_applicable(): ", model.is_lrp_applicable())

model.is_cam_applicable():  True
model.is_lrp_applicable():  True


### 2. User composites a project based on the registered inputs

#### Project

By the same way in input cases, user creates a project.

In [22]:
project, created = Project.objects.get_or_create(name="first_project")

In [23]:
Project.objects.all()

[<Project: first_project>]

Then, we can find empty spaces for datasets and models in the project. A project object has managers for its related datasets and models, which means `project.datasets` and `project.models` are also "managers".

In [24]:
print("project: ", project)
print("project.id: ", project.id)
print("project.name: ", project.name)
print("project.datasets.all(): ", project.datasets.all())
print("project.models.all(): ", project.models.all())
print("created: ", created)

project:  <Project: first_project>
project.id:  0
project.name:  first_project
project.datasets.all():  []
project.models.all():  []
created:  True


#### Add model and dataset to the project

User now registers the inputs to the project using `add_dataset` and `add_model` method.

In [26]:
project.add_dataset(dataset)
project.add_model(model)

<Project: first_project>

Now we can find that the inputs are registered for the project, using the related managers.

In [27]:
print("project.datasets.all(): ", project.datasets.all())
print("project.models.all(): ", project.models.all())

project.datasets.all():  [<Dataset: first_dataset>]
project.models.all():  [<Model: first_model>]


### 3. User composites tasks (experiments) based on the project inputs

A project has also task manager who controls its related tasks. It is empty not yet.

In [28]:
# no tasks written in database not yet
project.tasks.all()

[]

However, a single possible task can be defined based on the registered inputs.

In [29]:
# but some tasks can be defined based on dataset and model added
# in this case, a single task can be defined
project.list_all_task_data()

[(<Dataset: first_dataset>, <Model: first_model>)]

User registers all possible tasks by using `update_tasks` method.

In [30]:
# based on the added, create tasks and diretory for each task
project.update_tasks()

Then, the task is written in the database.

In [31]:
project.tasks.all()

[<Task: task_00_00>]

And user can get the task under the project easily.

In [32]:
task = project.tasks.all()[0]

In [34]:
task.to_dict()

{'id': 0,
 'name': 'task_00_00',
 'project_id': 0,
 'dataset_id': 0,
 'model_id': 0}

### 4. Explain

Finally, user gets explanations by all possible explainers.

In [35]:
# set common inputs for explainers
x_batch, y_batch = dataset.load_random_samples(n_samples=16)
p_batch = model.load()(x_batch).argmax(1)

/home/gkim/Projects/pnpxai-demo/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [40]:
# get all applicable explanations
explanations = task.run_all_applicables(x_batch, p_batch)

Running GradCAM:  58%|█████▊    | 7/12 [01:27<01:06, 13.27s/it]                  /home/gkim/Projects/pnpxai-demo/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
Running LayerCAM: 100%|██████████| 12/12 [01:51<00:00,  9.25s/it]       


Now, we can find the results are logged in "./xaistore".

### 5. API (future works)

All workflows can be consistently converted to API components using the objects and their managers. For example, a project API in `Flask` may look like

```python
from flask_restx import Resource, Api

app = Flask(__name__)
api = Api(app)

@api.route("/api/projects")
class ProjectPost(Resource):
    def post(self):
        proj, created = Projects.objects.get_or_create(name=request.json.get("data"))
        if created:
            return Response(proj.to_json(), status=201)
        return Response(status=403)

@api.route("/api/projects/<int:proj_id>")
class Projects(Resource):
    def get_obj(self, proj_id):
        proj = Project.objects.get(id=proj_id)
        if proj:
            return proj
        return Response(status=403)

    def get(self, proj_id):
        proj = self.get_obj(proj_id=proj_id)
        return Response(proj.to_json(), status=200)
    
    def put(self, proj_id):
        proj = self.get_obj(proj_id=proj_id)
        proj.name = request.json.get("data")
        proj.save()
        return Response(proj.to_json(), status=204)
    
    def delete(self, proj_id):
        proj = self.get_obj(proj_id=proj_id)
        proj.delete()
        return Response(status=204)

# ...
```